In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWj8ZhApmJ_VWd2xwlYYAKqPj4zcxhF25pk1I-2Wbdp1C4SN2zlZt-s
Mounted at /content/gdrive


In [ ]:
df= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/BDA_project/Data.csv',encoding = "ISO-8859-1")
df.head

<bound method NDFrame.head of             Date  ...                                              Top25
0     2000-01-03  ...                                 Recovering a title
1     2000-01-04  ...                       Millennium bug fails to bite
2     2000-01-05  ...                                       Useful links
3     2000-01-06  ...                        Lessons of law's hard heart
4     2000-01-07  ...                                         Gone aloft
...          ...  ...                                                ...
4096  2016-06-27  ...  Pakistani clerics declare transgender marriage...
4097  2016-06-28  ...  Mexico's security forces routinely use 'sexual...
4098  2016-06-29  ...  Tiny New Zealand town with 'too many jobs' lau...
4099  2016-06-30  ...  Law Enforcement Sources: Gun Used in Paris Ter...
4100  2016-07-01  ...  Ozone layer hole seems to be healing - US &amp...

[4101 rows x 27 columns]>

In [ ]:
train = df[df['Date'] < '20150101']
test = df[df['Date'] > '20141231']
train.shape

(3975, 27)

**Data preprocessing and cleaning**

In [ ]:
#Removing punctuations 
data=train.iloc[:,2:27]
data.replace("[^a-zA-Z]", " ",regex=True, inplace=True)

In [ ]:
data.head

<bound method NDFrame.head of                                                    Top1  ...                                              Top25
0     a  hindrance to operations   extracts from the...  ...                                 recovering a title
1                                             scorecard  ...                       millennium bug fails to bite
2                     coventry caught on counter by flo  ...                                       useful links
3                         pilgrim knows how to progress  ...                        lessons of law s hard heart
4                                  hitches and horlocks  ...                                         gone aloft
...                                                 ...  ...                                                ...
3970  brazil declares emergency after       babies a...  ...  cameron  osborne and murdoch back together at ...
3971  germany recruits       teachers to teach germa...  ...  experiment i

**Rename the column names**

In [ ]:
a=[i for i in range(25)]
index=[str(i) for i in (a)]
data.columns=index
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,a hindrance to operations extracts from the...,scorecard,hughes instant hit buoys blues,jack gets his skates on at ice cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,derby raise a glass to strupar s debut double,southgate strikes leeds pay the penalty,hammers hand robson a youthful lesson,saints party like it s,wear wolves have turned into lambs,stump mike catches testy gough s taunt,langer escapes to hit,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl s successor drawn into scandal,the difference between men and women,sara denver nurse turned solicitor,diana s landmine crusade put tories in a panic,yeltsin s resignation caught opposition flat f...,russian roulette,sold out,recovering a title
1,scorecard,the best lake scene,leader german sleaze inquiry,cheerio boyo,the main recommendations,has cubie killed fees,has cubie killed fees,has cubie killed fees,hopkins furious at foster s lack of hannibal...,has cubie killed fees,a tale of two tails,i say what i like and i like what i say,elbows eyes and nipples,task force to assess risk of asteroid collision,how i found myself at last,on the critical list,the timing of their lives,dear doctor,irish court halts ira man s extradition to nor...,burundi peace initiative fades after rebels re...,pe points the way forward to the ecb,campaigners keep up pressure on nazi war crime...,jane ratcliffe,yet more things you wouldn t know without the ...,millennium bug fails to bite
2,coventry caught on counter by flo,united s rivals on the road to rio,thatcher issues defence before trial by video,police help smith lay down the law at everton,tale of trautmann bears two more retellings,england on the rack,pakistan retaliate with call for video of walsh,cullinan continues his cape monopoly,mcgrath puts india out of their misery,blair witch bandwagon rolls on,pele turns up heat on ferguson,party divided over kohl slush fund scandal,manchester united england,women in record south pole walk,vasco da gama brazil,south melbourne australia,necaxa mexico,real madrid spain,raja casablanca morocco,corinthians brazil,tony s pet project,al nassr saudi arabia,ideal holmes show,pinochet leaves hospital after tests,useful links
3,pilgrim knows how to progress,thatcher facing ban,mcilroy calls for irish fighting spirit,leicester bin stadium blueprint,united braced for mexican wave,auntie back in fashion even if the dress look...,shoaib appeal goes to the top,hussain hurt by shambles but lays blame on e...,england s decade of disasters,revenge is sweet for jubilant cronje,our choice not theirs,profile of former us nazi party officer willia...,new evidence shows record of war crimes suspec...,the rise of the supernerds,written on the body,putin admits yeltsin quit to give him a head s...,bbc worst hit as digital tv begins to bite,how much can you pay for,christmas glitches,upending a table chopping a line and scoring ...,scientific evidence unreliable defence claims,fusco wins judicial review in extradition case,rebels thwart russian advance,blair orders shake up of failing nhs,lessons of law s hard heart
4,hitches and horlocks,beckham off but united survive,breast cancer screening,alan parker,guardian readers are you all whingers,hollywood beyond,ashes and diamonds,whingers a formidable minority,alan parker part two,thuggery toxins and ties,met faces fresh attack on race crime,everton fans top racist league of shame,our breasts ourselves,russia s new boss has an extremely strange his...,always and forever,most everywhere udis,most wanted chloe lunettes,return of the cane completely off the agenda,from sleepy hollow to greeneland,blunkett outlines vision for over s,embattled dobson attacks play now pay later ...,doom and the dome,what is the north south divide,aitken released from jai

**Converting to lower case sentences**

In [ ]:
for i in index:
    data[i]=data[i].str.lower()
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,a hindrance to operations extracts from the...,scorecard,hughes instant hit buoys blues,jack gets his skates on at ice cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,derby raise a glass to strupar s debut double,southgate strikes leeds pay the penalty,hammers hand robson a youthful lesson,saints party like it s,wear wolves have turned into lambs,stump mike catches testy gough s taunt,langer escapes to hit,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl s successor drawn into scandal,the difference between men and women,sara denver nurse turned solicitor,diana s landmine crusade put tories in a panic,yeltsin s resignation caught opposition flat f...,russian roulette,sold out,recovering a title
1,scorecard,the best lake scene,leader german sleaze inquiry,cheerio boyo,the main recommendations,has cubie killed fees,has cubie killed fees,has cubie killed fees,hopkins furious at foster s lack of hannibal...,has cubie killed fees,a tale of two tails,i say what i like and i like what i say,elbows eyes and nipples,task force to assess risk of asteroid collision,how i found myself at last,on the critical list,the timing of their lives,dear doctor,irish court halts ira man s extradition to nor...,burundi peace initiative fades after rebels re...,pe points the way forward to the ecb,campaigners keep up pressure on nazi war crime...,jane ratcliffe,yet more things you wouldn t know without the ...,millennium bug fails to bite
2,coventry caught on counter by flo,united s rivals on the road to rio,thatcher issues defence before trial by video,police help smith lay down the law at everton,tale of trautmann bears two more retellings,england on the rack,pakistan retaliate with call for video of walsh,cullinan continues his cape monopoly,mcgrath puts india out of their misery,blair witch bandwagon rolls on,pele turns up heat on ferguson,party divided over kohl slush fund scandal,manchester united england,women in record south pole walk,vasco da gama brazil,south melbourne australia,necaxa mexico,real madrid spain,raja casablanca morocco,corinthians brazil,tony s pet project,al nassr saudi arabia,ideal holmes show,pinochet leaves hospital after tests,useful links
3,pilgrim knows how to progress,thatcher facing ban,mcilroy calls for irish fighting spirit,leicester bin stadium blueprint,united braced for mexican wave,auntie back in fashion even if the dress look...,shoaib appeal goes to the top,hussain hurt by shambles but lays blame on e...,england s decade of disasters,revenge is sweet for jubilant cronje,our choice not theirs,profile of former us nazi party officer willia...,new evidence shows record of war crimes suspec...,the rise of the supernerds,written on the body,putin admits yeltsin quit to give him a head s...,bbc worst hit as digital tv begins to bite,how much can you pay for,christmas glitches,upending a table chopping a line and scoring ...,scientific evidence unreliable defence claims,fusco wins judicial review in extradition case,rebels thwart russian advance,blair orders shake up of failing nhs,lessons of law s hard heart
4,hitches and horlocks,beckham off but united survive,breast cancer screening,alan parker,guardian readers are you all whingers,hollywood beyond,ashes and diamonds,whingers a formidable minority,alan parker part two,thuggery toxins and ties,met faces fresh attack on race crime,everton fans top racist league of shame,our breasts ourselves,russia s new boss has an extremely strange his...,always and forever,most everywhere udis,most wanted chloe lunettes,return of the cane completely off the agenda,from sleepy hollow to greeneland,blunkett outlines vision for over s,embattled dobson attacks play now pay later ...,doom and the dome,what is the north south divide,aitken released from jai

In [22]:
data.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3965, 3966, 3967, 3968, 3969, 3970, 3971, 3972, 3973, 3974],
           dtype='int64', length=3975)

It returns the row numbers

**Appending 0 to 25 rows as a single sentence**

In [23]:
headlines=[]

for i in range(0,len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[i,0:25]))

In [24]:
headlines[0]

'a  hindrance to operations   extracts from the leaked reports scorecard hughes  instant hit buoys blues jack gets his skates on at ice cold alex chaos as maracana builds up for united depleted leicester prevail as elliott spoils everton s party hungry spurs sense rich pickings gunners so wide of an easy target derby raise a glass to strupar s debut double southgate strikes  leeds pay the penalty hammers hand robson a youthful lesson saints party like it s      wear wolves have turned into lambs stump mike catches testy gough s taunt langer escapes to hit     flintoff injury piles on woe for england hunters threaten jospin with new battle of the somme kohl s successor drawn into scandal the difference between men and women sara denver  nurse turned solicitor diana s landmine crusade put tories in a panic yeltsin s resignation caught opposition flat footed russian roulette sold out recovering a title'

**Model creation**

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

**Bag of words creation**

In [26]:
countvector=CountVectorizer(ngram_range=(2,2))
traindata=countvector.fit_transform(headlines)

In [27]:
type(traindata)

scipy.sparse.csr.csr_matrix

**Random forest classifier**

In [28]:
random_for=RandomForestClassifier(n_estimators=200,criterion='entropy')
random_for.fit(traindata,train['Label'])

RandomForestClassifier(criterion='entropy', n_estimators=200)

**Prediction for test data**

In [30]:
test_data=[]
for i in range(0,len(test.index)):
    test_data.append(' '.join(str(x) for x in test.iloc[i,2:27]))

test_data=countvector.transform(test_data)
test_pred=random_for.predict(test_data)

**Confusion matrix**

In [31]:
from sklearn.metrics import confusion_matrix
con=confusion_matrix(test_pred,test['Label'])
con

array([[138,   8],
       [ 48, 184]])

**Accuracy score**

In [32]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(test_pred,test['Label'])
acc*100

85.18518518518519

**Classification report**

In [33]:
from sklearn.metrics import classification_report
rep=classification_report(test_pred,test['Label'])
print(rep)

              precision    recall  f1-score   support

           0       0.74      0.95      0.83       146
           1       0.96      0.79      0.87       232

    accuracy                           0.85       378
   macro avg       0.85      0.87      0.85       378
weighted avg       0.87      0.85      0.85       378



**Taking few samples to test whether model is working good**

In [47]:
sen=['The stock prices fall dramatic']

In [35]:
ab=['The stock prices decrease due to wrap up of lockdown']

In [48]:
data=countvector.transform(sen)

In [49]:
pred=random_for.predict(data)

In [50]:
pred

array([1])

Thus the model is predicting very **good**